In [ ]:
import os
import sys
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import math
import numpy as np
import pandas as pd
import datetime
import pytz
import joblib
from sklearn.linear_model import LinearRegression
from matplotlib import pyplot

In [ ]:
# Auto-reload to directly apply changes 
%load_ext autoreload
%autoreload 2

# Display all columns
pd.set_option("display.max_columns", None)

# Import data



In [ ]:
# Load data
df = pd.read_csv("C://user/awesome_datascientist/myproject3/mvp/data/input_data.csv")
df = df.set_index("data_index_")

# Set timezone
df.index = pd.to_datetime(df.index).tz_convert("Europe/Amsterdam")

# Data exploration

In [ ]:
# Visualise results
def plot_figure_load(df):
    fig = go.Figure(
        make_subplots(
            shared_xaxes=True, vertical_spacing=0.02,
        )
    )
    fig.add_trace(
        go.Scatter(
            x=df.index,
            y=df["load_actuals_mw"],
            name="Actual",
            legendgroup="Actual",
            showlegend=True,
            line_color="green",
            opacity=0.5,
        ))
    title_text = "Energy demand"
    fig.update_layout(title={'text': title_text,
                             'y': 0.95,
                             'x': 0.5,
                             'xanchor': 'center',
                             'yanchor': 'top'},
                      autosize=False,
                      width=800,
                      height=800,
                      paper_bgcolor='white',
                      plot_bgcolor='white'
                      )

    fig.update_yaxes(title_text="Demand [MW]", title_standoff=30, title_font=dict(size=12),
                     showgrid=True, gridcolor='lightgrey',
                     zeroline=True, zerolinecolor='lightgrey',
                     )
    fig.update_xaxes(title_text="Date",
                     showgrid=True, gridcolor='lightgrey',
                     zeroline=True, zerolinecolor='lightgrey',
                     )

    return fig

In [ ]:
fig = plot_figure_load(df)
fig.show()

# 1. Batch model

## 1A. Baseline model

Because demand has a strong daily and weekly pattern, the baseline model is a seasonal naive forecats. The predicted demand is the demand at the same time of the day 1 week ago.

In [ ]:
# Baseline model: Shifted by 1 week
df_load_baseline = df.copy(deep=True).reset_index()[["load_actuals_mw"]]

df_load_baseline["data_index_"] = pd.to_datetime(
  df.reset_index()["data_index_"]
) + datetime.timedelta(weeks=1)

df_load_baseline = df_load_baseline.set_index("data_index_").rename(
    columns={"load_actuals_mw": "baseline"}
)

In [ ]:
# Plot next to each other
fig = plot_figure_load(df)

# Plot baseline prediction
fig.add_trace(
    go.Scatter(
        x=df_load_baseline.index,
        y=df_load_baseline["baseline"],
        name="Baseline model",
        legendgroup="Baseline model",
        showlegend=True,
        line_color="orange",
        opacity=0.5,
    ),
    col=1,
    row=1,
)

## 1B. ML model

In [ ]:
def add_fourier_features(df, column_name, period, n, period_name = "f"):
    t = df[column_name]
    for i in range(n):
        j = math.ceil((i+1)/2)
        if i%2:
            df[f'{period_name}_{i}'] = np.cos(j * 2 * np.pi * t / period)
        else:
            df[f'{period_name}_{i}'] = np.sin(j * 2 * np.pi * t / period)
    return df

def create_workday_weekend_features(df, fourier_order):
    # split features in workday / weekend
    df['is_workday'] = (~(df.is_holiday.astype(bool) | (df.day_of_week == 5) | (df.day_of_week == 6)))
    workday_data = {
        f'workday_{k}':df[k]*df.is_workday.astype(int)
        for k
        in ['temperature', 'solar_ghi'] + [f'f_quarter_{f}' for f in range(fourier_order)]
    }
    weekend_data = {
        f'weekend_{k}':df[k]*(~df.is_workday).astype(int)
        for k
        in ['temperature', 'solar_ghi'] + [f'f_quarter_{f}' for f in range(fourier_order)]
    }
    return workday_data, weekend_data

# add Fourier features to capture daily pattern in model
fourier_order = 6

df = add_fourier_features(df, "quarter_of_day", 4 * 24, fourier_order, "f_quarter")

# split workdays and weekend/holidays
workday_data, weekend_data = create_workday_weekend_features(df, fourier_order)
df_linregr = pd.DataFrame(
    {**workday_data, **weekend_data, "load": df["load_actuals_mw"]}
)
# List the input feature columns
feat_columns = list(workday_data.keys()) + list(weekend_data.keys())

In [ ]:
# Define size of train and test set of model
number_of_training_days = 30
number_of_test_days = 30

test_start_date_run_i = df_linregr.index.min() + datetime.timedelta(
    days=number_of_training_days
)
test_end_date = df_linregr.index.max()
df_result = pd.DataFrame()

# Run model for full period of data set
while True:
    print(f"Start of prediction of this run: {test_start_date_run_i}")

    # split train/test set
    df_train = df_linregr[
        test_start_date_run_i
        - datetime.timedelta(days=number_of_training_days) : test_start_date_run_i
    ]

    df_test = df_linregr[
        test_start_date_run_i : test_start_date_run_i
        + datetime.timedelta(days=number_of_test_days)
    ]

    lr = LinearRegression()
    lr.fit(df_train[feat_columns], df_train["load"])
    
    y_pred = lr.predict(df_test[feat_columns])
    
    # Combine results
    df_result_run_i = pd.DataFrame(
        {
            "load": df_test["load"],
            "pred": y_pred,
        }
    )

    # Store results in a single dataframe
    df_result = df_result.append(df_result_run_i)
    # Adjust start date of test set for next run
    test_start_date_run_i = test_start_date_run_i + datetime.timedelta(
        days=number_of_test_days
    )
    if test_start_date_run_i > test_end_date:
        break 


In [ ]:
# Plot next to each other
fig = plot_figure_load(df)

# Plot baseline prediction
fig.add_trace(
    go.Scatter(
        x=df_result.index,
        y=df_result["pred"],
        name="Linear regression model",
        legendgroup="Linear regression model",
        showlegend=True,
        line_color="magenta",
        opacity=0.5,
    ),
    col=1,
    row=1,
)

# 2. Real time model

Deploying your model real-time will beat even advanced ML batch models in performance. Therefore the model is kept simple.

In [ ]:
# Baseline model: Shifted by 15 minutes
df_load_rt = df.copy(deep=True).reset_index()[["load_actuals_mw"]]
df_load_rt["data_index_"] = pd.to_datetime(
    df.reset_index()["data_index_"]
) + datetime.timedelta(minutes=15)
df_load_rt = df_load_rt.set_index("data_index_").rename(
    columns={"load_actuals_mw": "baseline"}
)

In [ ]:
# Plot next to each other
fig = plot_figure_load(df)

# Plot baseline prediction
fig.add_trace(
    go.Scatter(
        x=df_load_rt.index,
        y=df_load_rt["baseline"],
        name="Baseline model",
        legendgroup="Baseline model",
        showlegend=True,
        line_color="orange",
        opacity=0.5,
    ),
    col=1,
    row=1,
)